In [40]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
import string
nltk.download('stopwords')
nltk.download('rslp')
! pip install unicodedata
from unicodedata import normalize


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dougl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\dougl\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
ERROR: Could not find a version that satisfies the requirement unicodedata (from versions: none)
ERROR: No matching distribution found for unicodedata


In [112]:
import pandas as pd
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import RegexpTokenizer
from unicodedata import normalize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from nltk.stem import RSLPStemmer

def open_dataset(dataset_name, dataset_type):
    if dataset_type == 'excel' or 'xlsx':
        df = pd.read_excel(str(dataset_name))
        return df
    elif dataset_type == 'csv':
        df = pd.read_excel(str(dataset_name))
        return df

def remove_characters(txt):
    sc = [c for c in txt.lower() if c not in string.punctuation]
    sc_ = ''.join(sc)
    return sc_

def remove_accents(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def tokenize(txt):
    return RegexpTokenizer('\w+').tokenize(txt)

def untokenize(txt):
    return (' ').join(i for i in txt if len(i)>1)

def remove_stopwords(txt):
    return ' '.join([w for w in txt.split() if w.lower() not in nltk.corpus.stopwords.words('portuguese')])

def stemming(txt):
    return ' '.join([nltk.stem.RSLPStemmer().stem(i) for i in txt.split()])

def simple_train(classifier_name, X, y):
    if classifier_name == 'Naive Bayes' or 'NB':
        clf = MultinomialNB().fit(X, y)
        return clf
    elif classifier_name == 'Support Vector Machine' or 'SVM':
        clf = SVC(gamma=1, C=1, kernel = 'rbf').fit(X, y)
        return clf
    elif classifier_name == 'K Nearest Neighbors' or 'KNN':
        clf = KNeighborsClassifier(n_neighbors = 3)
        return clf

def cv_train(classifier_name, X, y, n_fold=5):
    if classifier_name == 'Naive Bayes' or 'NB':
        clf = GridSearchCV(estimator  = MultinomialNB(),
                           param_grid = {'alpha': [0.001,0.01,0.1,1,10,100,1000],
                                         'fit_prior':[True, False]},
                           cv         = n_fold).fit(X, y)
        return clf
    
    elif classifier_name == 'Support Vector Machine' or 'SVM':    
        clf = GridSearchCV(estimator  = SVC(),
                           param_grid = {'C': [0.001,0.01,0.1,1,10,100,1000],
                                         'kernel': ['linear','poly','rbf','sigmoid'],
                                         'gamma': [0.001,0.01,0.1,1,10,100]},
                           cv         = n_fold).fit(X, y)
        return clf
    
    elif classifier_name == 'K Nearest Neighbors' or 'KNN':
        clf = GridSearchCV(estimator  = KNeighborsClassifier(),
                           param_grid = {'n_neighbors': [2,3,4,5,6],
                                         'algoritm':['auto','ball_tree','kd_tree','brute']},
                           cv         = n_fold).fit(X, y)
        return clf

def vectorizer(X, vec_type):
    if vec_type == 'countvectorizer':
        CountVec = CountVectorizer()
        numeric_matrix = CountVec.fit_transform(X)
        return numeric_matrix
    
    elif vec_type == 'tfidf-vectorizer' or 'tfidf':
        Tfidf = TfidfVectorizer()
        numeric_matrix = Tfidf.fit_transform(X)
        return numeric_matrix
    
def dataset_split(X, y, train_size):
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                       train_size   = train_size,
                                                       strify       = y,
                                                       random_state = 0,
                                                       shuffle      = True)
    return X_train, X_test, y_train, y_test

def confusion_matrix_analysis(y_test, list_predict):
    cm = []
    for i in range(len(list_predict)):
        cm.append(confusion_matrix(y_test, list_predict[i]))
    axis = sns.heatmap(cm/np.sum(cm),
                       annot = True,
                       fmt   = '.2%',
                       cmap  = 'Blues')
    return cm

def confusion_matrix_plot(models, X_test, y_test):
    fig, axis = plt.subplots(nrows   = 2,
                             ncols   = 2,
                             figsize = (15,10))
    for clf, ax in zip(models, axis.flatten()):
        plot_confusion_matrix(clf, X_test, y_test,
                              ax = axis, cmap= 'Blues',
                              display_labels=y_test)
        axis.title.set_text(type(clf).__name__)
    plt.tight_layout()
    plt.show()

In [ ]:
 def confusion_matrix_plot(y_test, list_predict):
        from sklearn.metrics import confusion_matrix
        cm_list = []
        for i in range(0, len(list_predict)):
            cm_list.append(confusion_matrix(y_test, list_predict[i]))
        
        return cm_list

    def accuracy(y_test, list_predict):
        from sklearn.metrics import accuracy_score
        accuracy = []
        for i in range(0, len(list_predict)):
            accuracy.append(int(accuracy_score(y_test, list_predict[i])100))
        return accuracy

    def fscore(y_test, list_predict):
        from sklearn.metrics import f1_score
        fscore = []
        for i in range(0, len(list_predict)):
            fscore.append(int(f1_score(y_test, list_predict[i], average = 'macro')100))
        return fscore

    def recall(y_test, list_predict):
        from sklearn.metrics import recall_score
        recall = []
        for i in range(0, len(list_predict)):
            recall.append(int(recall_score(y_test, list_predict[i], average = 'macro')100))
        return recall

    def precision(y_test, list_predict):
        from sklearn.metrics import precision_score
        precision = []
        for i in range(0, len(list_predict)):
            precision.append(int(precision_score(y_test, list_predict[i], average = 'macro')100))
        return precision

    def get_proba(X_test, list_model):
        proba_values = []
        for i in range(0, len(list_model)):
            proba_values.append(list_model[i].predict_proba(X_test))
        return proba_values